In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings,CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    temperature=0.1
)

memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

cache_dir = LocalFileStore("./.cache/")

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    separators="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("../files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)
vectorstore = FAISS.from_documents(
    docs, cached_embeddings
)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({ "input": question}, {"output": result.content})
    print(result)

invoke_chain("Is Aaronson guilty?")

content='According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.' response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 2243, 'total_tokens': 2266}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}


In [7]:
invoke_chain("What message did he write in the table?")

content='He wrote "2+2=5" in the dust on the table.' response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 2274, 'total_tokens': 2290}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}


In [8]:
invoke_chain("Who is Julia?")

content='Julia is a character in the document who was involved with Winston in acts of rebellion against the Party.' response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 2256, 'total_tokens': 2277}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}
